# Running the TTGammaProcessor

This cell will copy the test files from their location on eos to your local area. This only needs to be done once!

In [ ]:
%load_ext autoreload
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import time

List of samples to be run on (fileset variable) and a dictionary containing the number of events processed for each sample

In [ ]:
fileset = {
    "TTGamma_SingleLept": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2020/long_exercises/TTGamma/TestFiles/TTGamma_1l.root"
    ],
    "TTbarPowheg_Semilept": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTbar_1l.root"
    ],
    "W4jets": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2022/long_exercises/TTGamma/TestFiles/W4Jets.root"
    ],
    "WGamma_01J_5f": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2023/long_exercises/TTGamma/TestFiles/WGamma.root"
    ],
    "ZGamma_01J_5f_lowMass": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2024/long_exercises/TTGamma/TestFiles/ZGamma.root"
    ],
}

Run the TTGammaProcessor on the list of files included in fileset.

You can specify the chunksize and maximum number of chunks to process from each sample (selecting a small number of events and one chunk will force coffea to process only a subset of the events for quicker debugging)

In [ ]:
#autoreload forces the kernel to reload the processor to include any new changes
%autoreload 2
from ttgamma import TTGammaProcessor

tstart = time.time()

#Run Coffea code using uproot
iterative_run = processor.Runner(
    executor = processor.IterativeExecutor(compression=None),
    schema=NanoAODSchema,
    chunksize=40_000,
    maxchunks=1,
)
output = iterative_run(
    fileset,
    treename="Events",
    processor_instance=TTGammaProcessor(isMC=True),
)

elapsed = time.time() - tstart
print("Total time: %.1f seconds"%elapsed)

In [ ]:
output

In [ ]:
import hist
import matplotlib.pyplot as plt


fig, ax = plt.subplots()

for ds, histos in output.items():
    h = histos["all_photon_pt"]
    # h = histos["photon_chIso"]
    # for more complex histos you will have to reduce the dimensions with slicing
    # see https://uhi.readthedocs.io/en/latest/indexing.html for a guide
    # h = h[{"category": sum, "lepFlavor": sum, "systematic": "nominal"}]
    h.plot1d(ax=ax, label=ds)

ax.set_yscale("log")
ax.legend()

In [ ]:
expected = util.load("output_Expected.coffea")

hname = "photon_chIso"
for ds, histo in expected.items():
    mine = output[ds][hname].values()
    other = expected[ds][hname].values()
    if mine.shape != other.shape:
        print(f"For dataset {ds} there is a mismatch in axes")
    elif not np.all(mine == other):
        print(f"For dataset {ds} there is a mismatch in bin yields")
    else:
        print(f"All good for {ds}")

print("Done!")

# Accessing Arrays Interactively

Below is an example of loading a NanoAOD file interactively. This can be very useful for developing the code, and debugging any issues. Use this area to build your intuition for working with Coffea and awkward arrays!

In [ ]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

dataset = "TTGamma_SingleLept"
fname = fileset[dataset][0]
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema,
    entry_stop=40_000,
    metadata={"dataset": dataset},
).events()

Once you have opened the file, you can explore its contents using the 'fields' syntax

In [ ]:
events["Photon", "charge"] = 0
leadingMuon = ak.firsts(events.Muon)
leadingPhoton = ak.firsts(events.Photon)
leadingElectron = ak.firsts(events.Electron)

In [ ]:
(leadingMuon + leadingPhoton).mass

In [ ]:
mugammapairs = ak.cartesian({"mu":events.Muon, "gamma":events.Photon})
(mugammapairs.mu + mugammapairs.gamma).mass

In [ ]:
leadingMuon

In [ ]:
print(leadingMuon)

In [ ]:
type(leadingMuon)

In [ ]:
from ttgamma.scalefactors import mu_trig_err

mu_trig_err

In [ ]:
mu_trig_err(1.2, 36)

In [ ]:
events.Photon.matched_gen.fields

In [ ]:
events.GenPart.fields

There is also a docstring for each of these variables in NanoAOD, which you can access using '?':

In [ ]:
events.Jet.rawFactor?